# Sound anomaly detection
*Step 3 - Performing anomaly detection with a computer vision based approach, leveraging Amazon Rekognition Custom Labels*

## Introduction
---

In this notebook, we will use the spectrograms directly as inputs to feed a computer vision-based architecture. We will leverage Amazon Rekognition Custom Labels. Training a custom label project follows this process:
1. Building the datasets and uploading them to Amazon S3
2. Creating a project and collecting the generated project ARN
3. Associate the project with the training data, validation data and output locations
4. Train a project version with these datasets
5. Start the model: this will provision an endpoint and deploy the model behind it
6. Query the endpoint for inference for the validation and testing datasets

You need to ensure that this **notebook instance has an IAM role** which allows it to call the **Amazon Rekognition Custom Labels API**:
1. In your IAM console, look for the SageMaker execution role endorsed by your notebook instance (a role with a name like *AmazonSageMaker-ExecutionRole-yyyymmddTHHMMSS*)
2. Click on **Attach Policies** and look for this managed policy: **AmazonRekognitionCustomLabelsFullAccess**
3. Check the box next to it and click on **Attach Policy**

Your SageMaker notebook instance can now call the Rekognition Custom Labels APIs.

## Initialization
---

### Configuration

In [1]:
# Python libraries:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import sys
import time

# Other imports:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# AWS libraries:
import boto3

sys.path.append('tools')
import sound_tools
import utils
import rekognition_tools as rt

In [2]:
# Initialization:
%matplotlib inline
random.seed(42)
np.random.seed(42)
plt.style.use('Solarize_Light2')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# Paths definition:
DATA           = os.path.join('data', 'interim')
RAW_DATA       = os.path.join('data', 'raw')
PROCESSED_DATA = os.path.join('data', 'processed')
TRAIN_PATH     = os.path.join(PROCESSED_DATA, 'train')
TEST_PATH      = os.path.join(PROCESSED_DATA, 'test')

os.makedirs(os.path.join(PROCESSED_DATA, 'train', 'normal'), exist_ok=True)
os.makedirs(os.path.join(PROCESSED_DATA, 'train', 'abnormal'), exist_ok=True)
os.makedirs(os.path.join(PROCESSED_DATA, 'test', 'normal'), exist_ok=True)
os.makedirs(os.path.join(PROCESSED_DATA, 'test', 'abnormal'), exist_ok=True)

### Feature engineering parameters
These parameters are used to extract features from sound files:

In [ ]:
n_mels = 64
frames = 5
n_fft = 1024
hop_length = 512
power = 2.0

## **Step 1:** Building the datasets
---
### Generate list of sound files and splitting them
We are going to generate a spectrogram for each signal and use this as input to train a custom labels model with Rekognition:

* Testing dataset: **1110 signals** including:
  * 295 abnormal signals
  * 815 normal signals
* Training dataset: **4390 signals** including:
  * 1180 abnormal signals
  * 3210 normal signals

In [ ]:
# Build the list of normal and abnormal files:
normal_files, abnormal_files = utils.build_files_list(root_dir=os.path.join(DATA, 'fan'))

# Concatenate them to obtain a features and label datasets that we can split:
X = np.concatenate((normal_files, abnormal_files), axis=0)
y = np.concatenate((np.zeros(len(normal_files)), np.ones(len(abnormal_files))), axis=0)

train_files, test_files, train_labels, test_labels = train_test_split(X, y,
                                                                      train_size=0.8,
                                                                      random_state=42,
                                                                      shuffle=True,
                                                                      stratify=y
                                                                     )

### Generating spectrograms pictures

In [ ]:
img_train_files = sound_tools.generate_spectrograms(train_files, os.path.join(PROCESSED_DATA, 'train'))
img_test_files = sound_tools.generate_spectrograms(test_files, os.path.join(PROCESSED_DATA, 'test'))

### S3 buckets preparation
We upload the train and test dataset to S3 and generate the manifest files. **Update the BUCKET variable with your own Bucket name below**

In [ ]:
BUCKET = '<YOUR-BUCKET-NAME>'
PREFIX = 'custom-label'
LABELS = ['abnormal', 'normal']

In [ ]:
!aws s3 cp --recursive $TRAIN_PATH s3://$BUCKET/$PREFIX/train
!aws s3 cp --recursive $TEST_PATH s3://$BUCKET/$PREFIX/test

In [ ]:
rt.create_manifest_from_bucket(BUCKET, PREFIX, 'train', LABELS, output_bucket=f's3://{BUCKET}/{PREFIX}/manifests')
rt.create_manifest_from_bucket(BUCKET, PREFIX, 'test', LABELS, output_bucket=f's3://{BUCKET}/{PREFIX}/manifests')

## **Step 2:** Creating a custom label project in Amazon Rekognition
---

In [ ]:
# Initialization, get a Rekognition client:
PROJECT_NAME = 'sound-anomaly-detection'
reko = boto3.client("rekognition")

In [ ]:
# Let's try to create a Rekognition project:
try:
    project_arn = reko.create_project(ProjectName=PROJECT_NAME)['ProjectArn']
    
# If the project already exists, we get its ARN:
except reko.exceptions.ResourceInUseException:
    # List all the existing project:
    print('Project already exists, collecting the ARN.')
    reko_project_list = reko.describe_projects()
    
    # Loop through all the Rekognition projects:
    for project in reko_project_list['ProjectDescriptions']:
        # Get the project name (the string after the first delimiter in the ARN)
        project_name = project['ProjectArn'].split('/')[1]
        
        # Once we find it, we store the ARN and break out of the loop:
        if (project_name == PROJECT_NAME):
            project_arn = project['ProjectArn']
            break
            
project_arn

## **Step 3:** Associate the dataset to the project
---
We need to tell Rekognition where to find the training data, testing data and where to output its results

In [ ]:
TrainingData = {
    'Assets': [{ 
        'GroundTruthManifest': {
            'S3Object': { 
                'Bucket': BUCKET,
                'Name': f'{PREFIX}/manifests/train.manifest'
            }
        }
    }]
}

TestingData = {
    'AutoCreate': True
}

OutputConfig = { 
    'S3Bucket': BUCKET,
    'S3KeyPrefix': f'{PREFIX}/output'
}

## **Step 4:** Now we create a project version
---
Creating a project version will build and train a model within this Rekognition project for the data previously configured. Project creation can fail, if the bucket you selected cannot be accessed by Rekognition. Make sure the following Bucket Policy is applied to your bucket (replace **<YOUR-BUCKET-NAME>** by your bucket):

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AWSRekognitionS3AclBucketRead20191011",
            "Effect": "Allow",
            "Principal": {
                "Service": "rekognition.amazonaws.com"
            },
            "Action": [
                "s3:GetBucketAcl",
                "s3:GetBucketLocation"
            ],
            "Resource": "arn:aws:s3:::<YOUR-BUCKET-NAME>"
        },
        {
            "Sid": "AWSRekognitionS3GetBucket20191011",
            "Effect": "Allow",
            "Principal": {
                "Service": "rekognition.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:GetObjectAcl",
                "s3:GetObjectVersion",
                "s3:GetObjectTagging"
            ],
            "Resource": "arn:aws:s3:::<YOUR-BUCKET-NAME>/*"
        },
        {
            "Sid": "AWSRekognitionS3ACLBucketWrite20191011",
            "Effect": "Allow",
            "Principal": {
                "Service": "rekognition.amazonaws.com"
            },
            "Action": "s3:GetBucketAcl",
            "Resource": "arn:aws:s3:::<YOUR-BUCKET-NAME>"
        },
        {
            "Sid": "AWSRekognitionS3PutObject20191011",
            "Effect": "Allow",
            "Principal": {
                "Service": "rekognition.amazonaws.com"
            },
            "Action": "s3:PutObject",
            "Resource": "arn:aws:s3:::<YOUR-BUCKET-NAME>/*",
            "Condition": {
                "StringEquals": {
                    "s3:x-amz-acl": "bucket-owner-full-control"
                }
            }
        }
    ]
}
```

In [ ]:
version = 'experiment-1'
VERSION_NAME = f'{PROJECT_NAME}.{version}'

# Let's try to create a new project version in the current project:
try:
    project_version_arn = reko.create_project_version(
        ProjectArn=project_arn,      # Project ARN
        VersionName=VERSION_NAME,    # Name of this version
        OutputConfig=OutputConfig,   # S3 location for the output artefact
        TrainingData=TrainingData,   # S3 location of the manifest describing the training data
        TestingData=TestingData      # S3 location of the manifest describing the validation data
    )['ProjectVersionArn']
    
# If a project version with this name already exists, we get its ARN:
except reko.exceptions.ResourceInUseException:
    # List all the project versions (=models) for this project:
    print('Project version already exists, collecting the ARN:', end=' ')
    reko_project_versions_list = reko.describe_project_versions(ProjectArn=project_arn)
    
    # Loops through them:
    for project_version in reko_project_versions_list['ProjectVersionDescriptions']:
        # Get the project version name (the string after the third delimiter in the ARN)
        project_version_name = project_version['ProjectVersionArn'].split('/')[3]

        # Once we find it, we store the ARN and break out of the loop:
        if (project_version_name == VERSION_NAME):
            project_version_arn = project_version['ProjectVersionArn']
            break
            
print(project_version_arn)
status = reko.describe_project_versions(
    ProjectArn=project_arn,
    VersionNames=[project_version_arn.split('/')[3]]
)['ProjectVersionDescriptions'][0]['Status']

The following loops prints the project version training status (`TRAINING_IN_PROGRESS`) until the model has been trained (`TRAINING_COMPLETE`): if it's already trained the model status will either be:
* `STOPPED`: the model is trained, but is not currently deployed
* `STARTED`: the model has been deployed behind an endpoint and is available to deliver inference (hourly costs are incurred)
* `STARTING`: deployment in progress
* `STOPPING`: stopping in progress

In [ ]:
# Loops while training of this project version is in progress:
while status == 'TRAINING_IN_PROGRESS':
    status = reko.describe_project_versions(
        ProjectArn=project_arn,
        VersionNames=[project_version_arn.split('/')[3]]
    )['ProjectVersionDescriptions'][0]['Status']

    print(status)
    time.sleep(60)
    
print(status)

## **Step 5:** Model starting
We now have a trained model, we need to start it to serve inferences: the following command put the model in a "hosted" state. This process takes a while, as in the background we are created a dedicated endpoint on which we will deploy our trained model to serve the predictions:

In [ ]:
rt.start_model(project_arn, project_version_arn, VERSION_NAME)

## **Step 6:** Model evaluation
---
We now have a live endpoint with our model ready to deliver its predictions.
### Apply model on a test dataset
Let's now get the predictions on the **test datasets**:

In [ ]:
import s3fs

test_results_filename = os.path.join(PROCESSED_DATA, f'results_rekognition_{PROJECT_NAME}-{version}.csv')
print(f'Looking for test results file: "{test_results_filename}"')

if os.path.exists(test_results_filename):
    print('Prediction file on the test dataset exists, loading them from disk')
    test_predictions = pd.read_csv(test_results_filename)
    
else:
    print('Predictions file on the test dataset does not exist, querying the endpoint to collect inference results...')
    predictions_ok = rt.get_results(project_version_arn, BUCKET, s3_path=f'{BUCKET}/{PREFIX}/test/normal', label='normal', verbose=True)
    predictions_ko = rt.get_results(project_version_arn, BUCKET, s3_path=f'{BUCKET}/{PREFIX}/test/abnormal', label='abnormal', verbose=True)

    print('\nWriting predictions for test set to disk.')
    test_predictions = pd.concat([predictions_ok, predictions_ko], axis='index')
    test_predictions = rt.reshape_results(test_predictions)
    test_predictions.to_csv(test_results_filename, index=None)
    print('Done.')

### Confusion matrix analysis

In [ ]:
df = utils.generate_error_types(test_predictions, normal_label='normal', anomaly_label='abnormal')
tp = df['TP'].sum()
tn = df['TN'].sum()
fn = df['FN'].sum()
fp = df['FP'].sum()

utils.print_confusion_matrix(confusion_matrix(test_predictions['Ground Truth'], test_predictions['Prediction']), class_names=['abnormal', 'normal']);

In [ ]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tp + tn) / (tp + tn + fp + fn)
f1_score = 2 * precision * recall / (precision + recall)

print(f"""Amazon Rekognition custom model metrics:
- Precision: {precision*100:.1f}%
- Recall: {recall*100:.1f}%
- Accuracy: {accuracy*100:.1f}%
- F1 Score: {f1_score*100:.1f}%""")

## Cleanup
---
We need to stop the running model as we will continue to incur costs while the endpoint is live:

In [ ]:
rt.stop_model(project_version_arn)